In [1]:
!pip install faster-whisper transformers edge-tts sounddevice numpy langchain google-search-results langchain_community transformers torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 1.9 MB/s eta 0:00:00


In [2]:
import asyncio
from faster_whisper import WhisperModel
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import edge_tts
import numpy as np
import io
from google.colab import output
from IPython.display import Audio, display

In [3]:
def transcribe_audio(audio):
    model = WhisperModel("small", device="cpu", compute_type="int8")
    segments, _ = model.transcribe(audio, vad_filter=True, vad_parameters=dict(min_silence_duration_ms=500))
    return " ".join([segment.text for segment in segments])

In [14]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

# Log in to Hugging Face and store your token
notebook_login()

In [15]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
def generate_response(prompt):
    # Load model directly, using authentication
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-Large-Instruct-2407", use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-Large-Instruct-2407", use_auth_token=True)

    input_text = f"You are a helpfull assisstant, provide a comprehensive answer to the user question given below:  {prompt}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    outputs = model.generate(input_ids, max_new_tokens=200, num_return_sequences=1, temperature=0.7, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

In [16]:
import nest_asyncio

nest_asyncio.apply()

def text_to_speech(text, voice="en-US-AriaNeural", pitch="+0Hz", rate="+0%"):
    output_file = "output.mp3"
    command = f'edge-tts --voice "{voice}" --pitch "{pitch}" --rate "{rate}" --text "{text}" --write-media {output_file}'
    subprocess.run(command, shell=True, check=True)

    with open(output_file, "rb") as f:
        audio_data = f.read()
    os.remove(output_file)
    return audio_data

# import asyncio
# import nest_asyncio

# # Apply the patch to allow nested event loops
# nest_asyncio.apply()

# def text_to_speech_sync(text, voice="en-US-AriaNeural", pitch="+0Hz", rate="+0%"):
#     return asyncio.get_event_loop().run_until_complete(text_to_speech_async(text, voice, pitch, rate))

In [17]:
def voice_assistant_pipeline(audio_file):
    # Step 1: Transcribe audio
    text = transcribe_audio(audio_file)
    print(f"Transcribed text: {text}")

    # Step 2: Generate response using LLM
    response = generate_response(text)
    print(f"Generated response: {response}")

    # Step 3: Convert response to speech
    audio_data = text_to_speech_sync(response, voice="en-GB-SoniaNeural", pitch="+2Hz", rate="+10%")
    display(Audio(audio_data, rate=24000, autoplay=True))

In [18]:
from google.colab import files
uploaded = files.upload()

Saving car.m4a to car (1).m4a


In [19]:
if uploaded:
    audio_file = list(uploaded.keys())[0]
    voice_assistant_pipeline(audio_file)
else:
    print("No file was uploaded.")

Transcribed text:  In detail explain how to drive a car for a beginner.


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/142k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/588k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

model-00008-of-00051.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00009-of-00051.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 4831.94 MB. The target location /root/.cache/huggingface/hub/models--mistralai--Mistral-Large-Instruct-2407/blobs only has 3726.35 MB free disk space.
  warnings.warn(


model-00010-of-00051.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

KeyboardInterrupt: 